## Multivariate Multi-step Time Series Forecasting using Stacked LSTM sequence to sequence Autoencoder in Tensorflow 2.0 / Keras

This article will see how to create a stacked sequence to sequence the LSTM model for time series forecasting in Keras/ TF 2.0. 

In Sequence to Sequence Learning, an RNN model is trained to map an input sequence to an output sequence. The input and output need not necessarily be of the same length. The seq2seq model contains two RNNs, e.g., LSTMs. They can be treated as an encoder and decoder. The encoder part converts the given input sequence to a fixed-length vector, which acts as a summary of the input sequence.

This fixed-length vector is called the context vector. The context vector is given as input to the decoder and the final encoder state as an initial decoder state to predict the output sequence. Sequence to Sequence learning is used in language translation, speech recognition, time series
forecasting, etc.

We will use the sequence to sequence learning for time series forecasting. We can use this architecture to easily make a multistep forecast. we will add two layers, a repeat vector layer and time distributed dense layer in the architecture.

A repeat vector layer is used to repeat the context vector we get from the encoder to pass it as an input to the decoder. We will repeat it for n-steps ( n is the no of future steps you want to forecast). The output received from the decoder with respect to each time step is mixed. The time distributed densely will apply a fully connected dense layer on each time step and separates the output for each timestep. The time distributed densely is a wrapper that allows applying a layer to every temporal slice of an input.

We will stack additional layers on the encoder part and the decoder part of the sequence to sequence model. By stacking LSTM’s, it may increase the ability of our model to understand more complex representation of our time-series data in hidden layers, by capturing information at different levels.

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import tensorflow as tf
from pyFTS.models.nonstationary import nsfts
from pyFTS.benchmarks import Measures
from pyFTS.benchmarks import Measures
import matplotlib.pyplot as plt
from pyFTS.common import Util
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score
import datetime
import statistics
import math
import os
import sys
import statistics
sys.path.append("/home/hugo/projetos-doutorado/mimo_emb_fts/src/")

from embfts.util.DataSetUtil import DataSetUtil
from embfts.util.StatisticsUtil import StatisticsUtil

In [2]:
data_set_util = DataSetUtil()
statisticsUtil_util = StatisticsUtil()

In [3]:
def cal_nrmse(rmse, y):
    x = max(y)-min(y)
    return (rmse/x)

In [4]:
df = pd.read_csv('/home/hugo/projetos-doutorado/mimo_emb_fts/data/HomeC.csv', sep=',')
df = df.drop(labels=['icon','summary','cloudCover', 'House overall [kW]', 'time'], axis=1)
#df = df.loc[0: : 10]
data = data_set_util.clean_dataset(df)
data = data_set_util.series_to_supervised_mimo(data, 1, 1)
data


/home/hugo/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (0,27) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,use [kW](t-1),gen [kW](t-1),Dishwasher [kW](t-1),Furnace 1 [kW](t-1),Furnace 2 [kW](t-1),Home office [kW](t-1),Fridge [kW](t-1),Wine cellar [kW](t-1),Garage door [kW](t-1),Kitchen 12 [kW](t-1),...,temperature(t),humidity(t),visibility(t),apparentTemperature(t),pressure(t),windSpeed(t),windBearing(t),precipIntensity(t),dewPoint(t),precipProbability(t)
1,0.932833,0.003483,0.000033,0.020700,0.061917,0.442633,0.124150,0.006983,0.013083,0.000417,...,36.14,0.62,10.00,29.26,1016.91,9.18,282.0,0.0000,24.40,0.00
2,0.934333,0.003467,0.000000,0.020717,0.063817,0.444067,0.124000,0.006983,0.013117,0.000417,...,36.14,0.62,10.00,29.26,1016.91,9.18,282.0,0.0000,24.40,0.00
3,0.931817,0.003467,0.000017,0.020700,0.062317,0.446067,0.123533,0.006983,0.013083,0.000433,...,36.14,0.62,10.00,29.26,1016.91,9.18,282.0,0.0000,24.40,0.00
4,1.022050,0.003483,0.000017,0.106900,0.068517,0.446583,0.123133,0.006983,0.013000,0.000433,...,36.14,0.62,10.00,29.26,1016.91,9.18,282.0,0.0000,24.40,0.00
5,1.139400,0.003467,0.000133,0.236933,0.063983,0.446533,0.122850,0.006850,0.012783,0.000450,...,36.14,0.62,10.00,29.26,1016.91,9.18,282.0,0.0000,24.40,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
503905,1.608867,0.003217,0.000033,0.114300,0.623283,0.041817,0.005217,0.008350,0.013433,0.000467,...,35.12,0.86,8.74,29.45,1011.49,6.72,186.0,0.0101,31.27,0.51
503906,1.601233,0.003183,0.000050,0.085267,0.642417,0.041783,0.005267,0.008667,0.013483,0.000467,...,35.12,0.86,8.74,29.45,1011.49,6.72,186.0,0.0101,31.27,0.51
503907,1.599333,0.003233,0.000050,0.104017,0.625033,0.041750,0.005233,0.008433,0.013433,0.000467,...,35.12,0.86,8.74,29.45,1011.49,6.72,186.0,0.0101,31.27,0.51
503908,1.924267,0.003217,0.000033,0.422383,0.637733,0.042033,0.004983,0.008467,0.012933,0.000533,...,35.12,0.86,8.74,29.45,1011.49,6.72,186.0,0.0101,31.27,0.51


## Model Architecture

E1D1 ==> Sequence to Sequence Model with one encoder layer and one decoder layer.

In [5]:
def create_lstm_model_d1(neurons, n_past, n_future, n_features):
    encoder_inputs = tf.keras.layers.Input(shape=(n_past, n_features))
    encoder_l1 = tf.keras.layers.LSTM(neurons, return_state=True)
    encoder_outputs1 = encoder_l1(encoder_inputs)

    encoder_states1 = encoder_outputs1[1:]
    decoder_inputs = tf.keras.layers.RepeatVector(n_future)(encoder_outputs1[0])

    decoder_l1 = tf.keras.layers.LSTM(neurons, return_sequences=True)(decoder_inputs,initial_state = encoder_states1)
    decoder_outputs1 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(n_features))(decoder_l1)

    model_e1d1 = tf.keras.models.Model(encoder_inputs,decoder_outputs1)

    return model_e1d1

E2D2 ==> Sequence to Sequence Model with two encoder layers and two decoder layers.

In [6]:
def create_lstm_model_d2(neurons, n_past, n_future, n_features):
    encoder_inputs = tf.keras.layers.Input(shape=(n_past, n_features))
    encoder_l1 = tf.keras.layers.LSTM(neurons,return_sequences = True, return_state=True)
    encoder_outputs1 = encoder_l1(encoder_inputs)
    encoder_states1 = encoder_outputs1[1:]
    encoder_l2 = tf.keras.layers.LSTM(neurons, return_state=True)
    encoder_outputs2 = encoder_l2(encoder_outputs1[0])
    encoder_states2 = encoder_outputs2[1:]
    #
    decoder_inputs = tf.keras.layers.RepeatVector(n_future)(encoder_outputs2[0])
    #
    decoder_l1 = tf.keras.layers.LSTM(neurons, return_sequences=True)(decoder_inputs,initial_state = encoder_states1)
    decoder_l2 = tf.keras.layers.LSTM(neurons, return_sequences=True)(decoder_l1,initial_state = encoder_states2)
    decoder_outputs2 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(n_features))(decoder_l2)
    #
    model_e2d2 = tf.keras.models.Model(encoder_inputs,decoder_outputs2)
    #
    return model_e2d2
    

In [7]:
def compile_lstm_model(epochs, model, X_train,y_train, X_test,y_test, batch_size, verbose):
    reduce_lr = tf.keras.callbacks.LearningRateScheduler(lambda x: 1e-3 * 0.90 ** x)
    model.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.Huber())
    history_model = model.fit(X_train,y_train,epochs=epochs,validation_data=(X_test,y_test),batch_size=batch_size,verbose=verbose,callbacks=[reduce_lr])
    return model, history_model

In [8]:
def plot_history_model(history_model):
    plt.plot(history_model.history['loss'])
    plt.plot(history_model.history['val_loss'])
    plt.title("Model Loss")
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend(['Train', 'Valid'])
    plt.show()

In [9]:
def sliding_window(data,n_windows,train_size,epochs,batch_size,verbose,str_model,neurons,n_past,n_future,n_features):

    result = {
         "window": [],
         "rmse": [],
         "mape": [],
         "mae": [],
         "r2": [],
         "smape": [],
         "nrmse": [],
         "variable":[]
    }
    
    final_result = {
         "window": [],
         "rmse": [],
         "mape": [],
         "mae": [],
         "r2": [],
         "smape": [],
         "nrmse": [],
         "variable":[]
    }

    tam = len(data)
    windows_length = math.floor(tam / n_windows)
    for ct, ttrain, ttest in Util.sliding_window(data, windows_length, train_size, inc=1):
        if len(ttest) > 0:
            
            print('-' * 20)
            print(f'training window {(ct)}')
            
            scaler = StandardScaler()

            Xtrain = scaler.fit_transform(ttrain.loc[:,'use [kW](t-1)':'precipProbability(t-1)'])
            ytrain = scaler.fit_transform(ttrain.loc[:,'use [kW](t)':'precipProbability(t)'])
            Xtest = scaler.transform(ttest.loc[:,'use [kW](t-1)':'precipProbability(t-1)'])
            ytest = scaler.transform(ttest.loc[:,'use [kW](t)':'precipProbability(t)'])
            
#             Xval = Xtest[:int(len(Xtest)*0.25)]
#             yval = ytest[:int(len(ytest)*0.25)]

            Xval = Xtrain[:int(len(Xtrain)*0.15)]
            yval = ytrain[:int(len(ytrain)*0.15)]
            
            X_train = Xtrain.reshape(Xtrain.shape[0], 1, Xtrain.shape[1])
            y_train = ytrain.reshape(ytrain.shape[0], 1, ytrain.shape[1])
            X_test = Xtest.reshape(Xtest.shape[0], 1, Xtest.shape[1])
            y_test = ytest.reshape(ytest.shape[0], 1, ytest.shape[1])
            X_val = Xval.reshape(Xval.shape[0], 1, Xval.shape[1])
            y_val = yval.reshape(yval.shape[0], 1, yval.shape[1])
            
            if str_model == 'd1':
                model = create_lstm_model_d1(neurons, n_past, n_future, n_features)
                model,history_model = compile_lstm_model(epochs,model,X_train,y_train,X_val,y_val,batch_size,verbose)
            else:
                model = create_lstm_model_d2(neurons, n_past, n_future, n_features)
                model,history_model = compile_lstm_model(epochs,model,X_train,y_train,X_val,y_val,batch_size,verbose)
             
            #plot_history_model(history_model)
            
            
            df_forecats_columns = ttest.loc[:,'use [kW](t)':'precipProbability(t)'].columns
            columns = list(df_forecats_columns)
            
            prediction = model.predict(X_test)
            prediction = prediction.reshape(prediction.shape[0], prediction.shape[2])
            prediction = scaler.inverse_transform(prediction)            
            df_forecast = pd.DataFrame(prediction,columns=columns)
            
            #ytest_metric = scaler.inverse_transform(y_test.reshape(y_test.shape[0], y_test.shape[2]))
            ytest_metric = ttest.loc[:,'use [kW](t)':'precipProbability(t)'].values
            df_original = pd.DataFrame(ytest_metric,columns=columns)
            
            for col in columns:  
                original = df_original[col].values
                forecast = df_forecast[col].values
#                 original = original[:len(original)-1]
#                 forecast = forecast[1:]
                
                mae = round(mean_absolute_error(original,forecast),3)
                r2 = round(r2_score(original,forecast),3)
                #rmse = mean_squared_error(original,forecast,squared=False)
                rmse = round(Measures.rmse(original,forecast),3)
                mape = round(Measures.mape(original,forecast),3)
                nrmse = round(cal_nrmse(rmse, original),3)
                smape = round(Measures.smape(original,forecast),3)

                result["rmse"].append(rmse)
                result["nrmse"].append(nrmse)
                result["mape"].append(mape)
                result["mae"].append(mae)
                result["r2"].append(r2)
                result["smape"].append(smape)
                result["window"].append(ct)
                result["variable"].append(col)
                
#                 fig, ax = plt.subplots(nrows=1, ncols=1, figsize=[15, 3])
#                 ax.plot(original, label='Original')
#                 ax.plot(forecast, label='Forecast')
#                 handles, labels = ax.get_legend_handles_labels()
#                 lgd = ax.legend(handles, labels, loc=2, bbox_to_anchor=(1, 1))
#                 plt.show()
        
    measures = pd.DataFrame(result)
    return measures

In [10]:
n_windows = 30
train_size = 0.75 
epochs = 25
batch_size = 32
verbose = 0
neurons = 100 
n_past = 1 
n_future = 1 
n_features = df.shape[1] # 27
str_model = 'd1'

model_d1_result = sliding_window(data,n_windows,train_size,epochs,batch_size,verbose,str_model,neurons,n_past,n_future,n_features)

--------------------
training window 0


/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWar

--------------------
training window 16796


/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWar

--------------------
training window 33592


...

/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWar

--------------------
training window 470288


/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWar

--------------------
training window 487084


/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWar

In [13]:
df_forecats_columns = data.loc[:,'use [kW](t)':'precipProbability(t)'].columns

columns = list(df_forecats_columns)

final_result = {
    "variable": [],
    "rmse": [],
    "mae": [],
    "mape": [],
    "r2": [],
    "smape": [],
    "nrmse": []
}

measures = model_d1_result
var = measures.groupby("variable")

for col in columns:
    
    var_agr = var.get_group(col)
           
    rmse = round(statistics.mean(var_agr.loc[:,'rmse']),3)
    mape = round(statistics.mean(var_agr.loc[:,'mape']),3)
    mae = round(statistics.mean(var_agr.loc[:,'mae']),3)
    r2 = round(statistics.mean(var_agr.loc[:,'r2']),3)
    smape = round(statistics.mean(var_agr.loc[:,'smape']),3)
    nrmse = round(statistics.mean(var_agr.loc[:,'nrmse']),3)

    final_result["variable"].append(col)
    final_result["rmse"].append(rmse)
    final_result["mape"].append(mape)
    final_result["mae"].append(mae)
    final_result["r2"].append(r2)
    final_result["smape"].append(smape)
    final_result["nrmse"].append(nrmse)
        
    #print(f'Results: {(col,rmse,mae,r2)}')
        
        
final_measures_d1 = round(pd.DataFrame(final_result),3)

In [14]:
final_measures_d1

,variable,rmse,mae,mape,r2,smape,nrmse
0,use [kW](t),0.391,0.188,90.327,0.609,11.740,0.060
1,gen [kW](t),0.035,0.018,inf,0.870,35.987,0.073
2,Dishwasher [kW](t),0.064,0.020,inf,-77135.581,94.003,8.047
3,Furnace 1 [kW](t),0.067,0.031,61.693,0.680,19.444,0.084
4,Furnace 2 [kW](t),0.067,0.035,30.027,0.040,10.494,0.125
5,Home office [kW](t),0.023,0.011,16.500,-2.092,8.042,0.105
6,Fridge [kW](t),0.027,0.012,113.027,0.851,24.470,0.050
7,Wine cellar [kW](t),0.014,0.006,35.083,0.907,14.629,0.073
8,Garage door [kW](t),0.012,0.002,7.772,0.086,3.373,0.034
9,Kitchen 12 [kW](t),0.007,0.002,inf,-302.371,38.536,0.515


In [15]:
final_measures_d1.to_csv (r'stacked_lstm_d1_kaggle_sh.csv', index = False, header=True)

In [16]:
n_windows = 30
train_size = 0.75 
epochs = 25
batch_size = 32
verbose = 0
neurons = 100 
n_past = 1 
n_future = 1 
n_features = df.shape[1] # 27
str_model = 'd2'

model_d2_result = sliding_window(data,n_windows,train_size,epochs,batch_size,verbose,str_model,neurons,n_past,n_future,n_features)

--------------------
training window 0


/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWar

--------------------
training window 16796


/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWar

--------------------
training window 33592


...

/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWar

--------------------
training window 470288


/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWar

--------------------
training window 487084


/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWar

In [19]:
df_forecats_columns = data.loc[:,'use [kW](t)':'precipProbability(t)'].columns

columns = list(df_forecats_columns)

final_result = {
    "variable": [],
    "rmse": [],
    "mae": [],
    "mape": [],
    "r2": [],
    "smape": [],
    "nrmse": []
}

measures = model_d2_result
var = measures.groupby("variable")

for col in columns:
    
    var_agr = var.get_group(col)
           
    rmse = round(statistics.mean(var_agr.loc[:,'rmse']),3)
    mape = round(statistics.mean(var_agr.loc[:,'mape']),3)
    mae = round(statistics.mean(var_agr.loc[:,'mae']),3)
    r2 = round(statistics.mean(var_agr.loc[:,'r2']),3)
    smape = round(statistics.mean(var_agr.loc[:,'smape']),3)
    nrmse = round(statistics.mean(var_agr.loc[:,'nrmse']),3)

    final_result["variable"].append(col)
    final_result["rmse"].append(rmse)
    final_result["mape"].append(mape)
    final_result["mae"].append(mae)
    final_result["r2"].append(r2)
    final_result["smape"].append(smape)
    final_result["nrmse"].append(nrmse)
        
    #print(f'Results: {(col,rmse,mae,r2)}')
        
        
final_measures_d2 = round(pd.DataFrame(final_result),3)

In [20]:
final_measures_d2

,variable,rmse,mae,mape,r2,smape,nrmse
0,use [kW](t),0.381,0.184,94.315,0.679,11.994,0.058
1,gen [kW](t),0.035,0.018,inf,0.874,36.525,0.073
2,Dishwasher [kW](t),0.065,0.019,inf,-70650.755,94.099,7.381
3,Furnace 1 [kW](t),0.068,0.032,65.574,0.680,20.271,0.085
4,Furnace 2 [kW](t),0.064,0.033,25.919,0.206,9.911,0.120
5,Home office [kW](t),0.025,0.012,17.585,-0.278,8.418,0.080
6,Fridge [kW](t),0.029,0.013,109.356,0.825,24.681,0.053
7,Wine cellar [kW](t),0.015,0.007,48.358,0.875,15.120,0.081
8,Garage door [kW](t),0.012,0.002,8.424,0.030,3.932,0.034
9,Kitchen 12 [kW](t),0.007,0.002,inf,-696.257,35.891,0.632


In [21]:
final_measures_d2.to_csv (r'stacked_lstm_d2_kaggle_sh.csv', index = False, header=True)